In [ ]:
import math
import pandas as pd

def m_ge(x,t,scale = 1.0):
    return (x-t)/scale

def m_l(x,t,scale = 1.0):
    return (t-x)/scale

def m_range(x,lo,hi,scale = 1.0):
    return min(x-lo, hi-x)/scale

